In [7]:
search_for = 92
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002691984176635742
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 75
all cases: 190457211
type: [1, 1, 1, 1, 92] 92
cases of this type: 71639296
10000 22.666563866461388
20000 22.310992226886334
30000 56.44617370885521
40000 92.68358495263324
50000 92.9865419269594
60000 84.81100922871441
70000 87.68931669870831
80000 100.12677242419977
90000 106.38702922176721
100000 118.3386187210372
110000 129.91592571873298
120000 126.04721826738918
130000 128.85928507742813
140000 128.8557665559363
150000 138.19479191376095
160000 139.8078203212694
170000 121.64088419420995
180000 118.98008991846999
190000 138.29899345038453
200000 146.244545928982
210000 140.69113997136105
220000 125.02203774152885
230000 131.2993206878073
240000 109.82802426242866
250000 110.90732065873414
260000 119.73452765515579
270000 107.8523752377588
280000 107.88521970805034
290000 122.69913906354469
300000 108.8634537288025
310000 106.4597448

3060000 91.08109136893063
3070000 104.60921646619978
3080000 56.70745586687996
3090000 38.45660229145318
3100000 39.575499637039506
3110000 39.913131165637125
3120000 78.56815494474951
3130000 130.28380175149348
3140000 47.19900115107574
3150000 32.60763158273015
3160000 33.01721164763073
3170000 32.142709569726506
3180000 85.4279202925045
3190000 118.08083644143223
3200000 93.00437570696515
3210000 101.26364052706438
3220000 107.12504072300803
3230000 100.17485438650215
3240000 104.44412720173356
3250000 97.4953782359128
3260000 98.467244816503
3270000 99.78779432699876
3280000 99.71323156331351
3290000 101.61268083233286
3300000 105.892831926531
3310000 100.35331954732088
3320000 101.49229254837049
3330000 105.63645359927683
3340000 106.22833210798181
3350000 99.35102040042645
3360000 96.48607802911222
3370000 96.1566719261512
3380000 96.02142746826843
3390000 91.68299663612106
3400000 94.15365518176702
3410000 98.0373495290213
3420000 94.25926545032306
3430000 99.58897046837588
3440

6180000 38.295399835340426
6190000 38.906738491720894
6200000 74.89329588428103
6210000 109.47845946851065
6220000 96.65767055794261
6230000 99.45222740019751
6240000 107.97925199454068
6250000 104.43805514161083
6260000 90.86783415627792
6270000 102.53476939277809
6280000 118.52986514024201
6290000 35.260606676641984
6300000 32.17703871941063
6310000 31.64793838569256
6320000 31.814801196221023
6330000 95.17089890700149
6340000 103.75485396496397
6350000 93.93403344134151
6360000 103.41468703007018
6370000 93.75653705453395
6380000 99.47180683555207
6390000 98.04050971703494
6400000 100.55053249603525
6410000 95.52056475073164
6420000 93.43002681766838
6430000 95.51840896384535
6440000 93.56290224836745
6450000 93.35384444415402
6460000 98.9017868958981
6470000 101.82512438410004
6480000 99.46568849513545
6490000 99.24071534588117
6500000 101.82605630628363
6510000 95.20084688313567
6520000 93.89014250652512
6530000 88.93761165983759
6540000 89.42773467473496
6550000 94.67489854371645

9310000 93.102576023576
9320000 95.28815080871739
9330000 110.77367072170158
9340000 102.7289692969767
9350000 104.69205141352992
9360000 116.6815442178413
9370000 98.85598828661402
9380000 96.21886882015093
9390000 110.81673094043578
9400000 91.64538458618763
9410000 94.63049684938593
9420000 106.65713856034286
9430000 104.95429396585281
9440000 31.090048096860365
9450000 30.98938584648244
9460000 30.891570646276108
9470000 36.13654599504222
9480000 103.0130397799266
9490000 92.7447131084454
9500000 103.93042031335965
9510000 96.06195058289988
9520000 99.68362681599108
9530000 100.40189613206843
9540000 112.7281870762816
9550000 92.0784798190758
9560000 102.07373345293905
9570000 94.91795136552237
9580000 91.04519813683007
9590000 101.14271277499954
9600000 107.68604662011417
9610000 97.50644462102979
9620000 92.58712225605564
9630000 95.88457823972492
9640000 86.92692297644267
9650000 82.31823140508956
9660000 92.41518573494537
9670000 84.78858446499271
9680000 81.41899116585515
9690

12360000 56.45807882958483
12370000 108.23033208850428
12380000 90.4110536019296
12390000 94.52722894295228
12400000 95.13676902818453
12410000 91.04210588340577
12420000 94.36529856663644
12430000 88.25508279020644
12440000 95.69991412211357
12450000 95.13489194565493
12460000 101.40093165501455
12470000 102.62006954582985
12480000 97.94549540001923
12490000 91.98928357949646
12500000 95.41713266378171
12510000 91.49977684010442
12520000 87.42850030041649
12530000 96.07093146582997
12540000 93.9971857856155
12550000 96.88091670181545
12560000 86.26041440987977
12570000 106.51777990018125
12580000 91.74905351774827
12590000 31.45539002187275
12600000 30.666708959198935
12610000 31.31876236542515
12620000 33.424561842268304
12630000 104.987150376962
12640000 92.40846568719464
12650000 87.79899008287911
12660000 95.29115560688483
12670000 91.24486828830528
12680000 93.08803574040734
12690000 87.84123785394145
12700000 89.91522137921531
12710000 88.14650677620224
12720000 96.2357370563561

...

177440000 0.8844658267440002
177450000 0.8900514880017637
177460000 0.8923010653765135
177470000 0.8889304765145051
177480000 1.1585649524896542
177490000 1.200948736709661
177500000 1.126292736630612
177510000 1.154514795891106
177520000 1.1858138303462598
177530000 1.120741280530042
177540000 1.1496636012298662
177550000 1.1473376654000018
177560000 1.1419178369406726
177570000 0.8858919381813606
177580000 0.8837203491576049
177590000 0.9179053916932675
177600000 0.9000582373527288
177610000 0.8835176497742732
177620000 0.8874433271645308
177630000 0.88695488537848
177640000 0.8865793639261986
best so far: 0
type: [1, 2, 1, 46, 1] 92
cases of this type: 194672
177650000 0.8747976796928578
177660000 0.8690943945128321
177670000 0.8813395048017104
177680000 0.8728560792190103
177690000 0.8657653652624487
177700000 0.8642427947189941
177710000 0.8648716777828799
177720000 0.8776152943575778
177730000 0.8944689231080545
177740000 1.1296537770668003
177750000 1.164802831991434
177760000 1

best so far: 0
type: [1, 2, 46, 1, 1] 92
cases of this type: 4232
180200000 0.6885760446172954
best so far: 0
type: [1, 4, 1, 1, 23] 92
cases of this type: 1119364
180210000 1.348936379846394
180220000 3.022622704833137
180230000 1.3498572947846914
180240000 2.7058201985179386
180250000 2.3445047260762117
180260000 1.7420168870341843
180270000 2.963803816112081
180280000 1.0036285338752733
180290000 2.490652828747034
180300000 1.9619778157671692
180310000 1.560555006805804
180320000 2.3199656623586016
180330000 0.8531788923596542
180340000 2.2317019248784518
180350000 2.049247881072475
180360000 1.7933035889925162
180370000 2.118493464866082
180380000 1.121965835110777
180390000 2.040907284610927
180400000 2.3693435139347185
180410000 1.7925201225534146
180420000 1.8428877963293395
180430000 1.507209075814459
180440000 1.9001783593892785
180450000 2.5826171309606436
180460000 1.8168229375784333
180470000 1.4697054063590766
180480000 1.532868818519652
180490000 1.7939354680942696
180500

182720000 1.94384904586339
182730000 1.3176101493051051
182740000 2.048909777475973
182750000 1.9480973130622574
182760000 2.3181194730992116
182770000 2.119763105216185
182780000 1.7698231585329107
182790000 1.4733658740160862
182800000 1.2959046997961798
182810000 1.4222040078981717
182820000 1.4316303923214673
182830000 1.6195002866817512
182840000 2.176734613282654
182850000 2.342806530417641
182860000 2.0845620630271835
182870000 1.9011589599035186
182880000 1.510444194598337
182890000 1.5346336160833902
182900000 0.5984686712918811
182910000 1.7103163573108913
182920000 1.197197571653366
182930000 1.9564979652023513
182940000 2.1562870489172736
182950000 2.0188438387115397
182960000 1.756498306684408
182970000 1.8295377641739847
182980000 1.5932194260580277
182990000 1.616046150201433
183000000 0.7594618219912649
183010000 1.5379140796417263
183020000 1.3851851629305019
183030000 2.002363645716667
183040000 1.8516589415210154
183050000 2.2665042418137986
183060000 1.7836963545101

185560000 1.1799650629180602
185570000 0.634504254969034
185580000 1.2237862844107745
185590000 0.927703783298605
185600000 1.0926207794084681
185610000 1.3031004259860517
185620000 1.0957947863172954
185630000 0.6301023403201037
185640000 1.1157372744114598
185650000 1.244258402873357
185660000 1.3091108059583572
185670000 0.6854038905194204
185680000 0.9026631723736657
185690000 1.4089450530440544
185700000 1.3121349318982958
185710000 1.1182611028491523
185720000 0.751419514235437
185730000 0.9622385806509057
185740000 1.1678003674114545
185750000 1.4278896147328086
185760000 1.292225514511605
185770000 0.678133907040351
185780000 0.9687763493134511
185790000 1.370675300082028
185800000 1.2318207528145313
185810000 1.3239438003327781
185820000 0.5946965267478228
185830000 1.2701640833809509
185840000 1.3754125185547736
185850000 1.0773967300680083
185860000 1.3177976667502522
best so far: 0
type: [2, 1, 1, 2, 23] 92
cases of this type: 2238728
185870000 0.7481625586075584
185880000 

188300000 0.1262965467877719
188310000 0.14871501161503792
188320000 0.13619765196308162
188330000 0.13384313170282047
188340000 0.09955375334660213
188350000 0.11330078412337434
188360000 0.11784189853243034
188370000 0.11605057811351617
188380000 0.13794892750045987
188390000 0.13477125470912457
best so far: 0
type: [2, 1, 2, 1, 23] 92
cases of this type: 1119364
188400000 0.18738188856035473
188410000 0.4000984595199956
188420000 0.3199435833969778
188430000 0.27549872822072113
188440000 0.3672463206852608
188450000 0.2575366923431423
188460000 0.37342748636279505
188470000 0.27455503978279566
188480000 0.29080344889239473
188490000 0.3119340784761906
188500000 0.30124233185137644
188510000 0.367378621817602
188520000 0.25684905079259474
188530000 0.30559767889847356
188540000 0.24191556273294823
188550000 0.31568694385554397
188560000 0.28680145365808407
188570000 0.2267442831227713
188580000 0.31397297107440236
188590000 0.2876966931573815
188600000 0.33652594622047743
188610000 0